# Import Library

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise import accuracy

# Import Data User, Anime, UserAnime

In [2]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))

# Mengkalikan Matriks User-Item dengan Status

Jika Rating[i] dan Status[i] tidak ada
maka Rating[i] * 2

In [3]:
ScoresDF

,username,anime_id,my_score,my_status,animes_rated
0,Tsundora,21,2,4,259
1,Tsundora,59,5,2,259
2,Tsundora,210,5,2,259
3,Tsundora,249,7,4,259
4,Tsundora,269,6,2,259
...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4
193268,Senpai_Helix,18677,9,2,4
193269,Kiyoshi-Kun,1722,10,2,3
193270,Kiyoshi-Kun,2993,10,2,3


In [4]:
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 2, 5, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 3, 2, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 6, 2, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 1, 3, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 4, 1, ScoresDF['my_status'])

In [5]:
ScoresDF.loc[ScoresDF['my_status'] == None]

,username,anime_id,my_score,my_status,animes_rated


In [6]:
ScoresDF['scoreXstatus'] = ScoresDF['my_score'] * ScoresDF['my_status']
ScoresDF['scoreXstatus'] = ScoresDF['scoreXstatus'] / 5
ScoresDF

,username,anime_id,my_score,my_status,animes_rated,scoreXstatus
0,Tsundora,21,2,1,259,0.4
1,Tsundora,59,5,5,259,5.0
2,Tsundora,210,5,5,259,5.0
3,Tsundora,249,7,1,259,1.4
4,Tsundora,269,6,5,259,6.0
...,...,...,...,...,...,...
193267,Senpai_Helix,4214,10,5,4,10.0
193268,Senpai_Helix,18677,9,5,4,9.0
193269,Kiyoshi-Kun,1722,10,5,3,10.0
193270,Kiyoshi-Kun,2993,10,5,3,10.0


# Membuat User-Item Matriks Berdasarkan Rating
Serta mengisi data yang kosong dengan 0

In [7]:
UserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='scoreXstatus')
UserItemRatingMatrix = UserItemRatingMatrix.replace(np.nan,0)
UserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training dan Test Model
Training dan test model dipisah 80% : 20%

In [8]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','scoreXstatus']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

# Tuning Parameter

In [13]:
svd_param = {
    'n_epochs' : [5,10,15,20,25],
    'lr_all' : [0.01],
    'reg_all' :[0.4]
}

knn_param = {
    'k' : [5,10,15,20,25,40,50],
    'sim_options':{
        'name': ['pearson_baseline'],
        'min_support' : [1],
        'user_based' : 'False',
    },
    'bsl_options': {
        'method': ['als'],
        'reg': [1]
    }
}

# gsSVD = GridSearchCV(SVD, svd_param, measures=['rmse','mae'], cv=3)
# gsSVD.fit(data)

gsKNN = GridSearchCV(KNNBaseline, knn_param, measures=['rmse','mae'], cv=3)
gsKNN.fit(data)

# print("Best Score SVD : ", gsSVD.best_score['rmse'])
# print("Best Param SVD : ", gsSVD.best_params['rmse'])

print("Best Score KNNBaseline : ", gsKNN.best_score['rmse'])
print("Best Param KNNBaseline : ", gsKNN.best_params['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [19]:
cv_result = gsKNN.cv_results
cv_result

{'split0_test_rmse': array([1.81888809, 1.81888809, 1.81888809, 1.81888809, 1.81888809,
        1.77046233, 1.77046233, 1.77046233, 1.77046233, 1.77046233,
        1.75678712, 1.75678712, 1.75678712, 1.75678712, 1.75678712,
        1.75134925, 1.75134925, 1.75134925, 1.75134925, 1.75134925,
        1.74884351, 1.74884351, 1.74884351, 1.74884351, 1.74884351,
        1.74727913, 1.74727913, 1.74727913, 1.74727913, 1.74727913,
        1.74734309, 1.74734309, 1.74734309, 1.74734309, 1.74734309]),
 'split1_test_rmse': array([1.81819131, 1.81819131, 1.81819131, 1.81819131, 1.81819131,
        1.76736897, 1.76736897, 1.76736897, 1.76736897, 1.76736897,
        1.75497553, 1.75497553, 1.75497553, 1.75497553, 1.75497553,
        1.74974797, 1.74974797, 1.74974797, 1.74974797, 1.74974797,
        1.74754122, 1.74754122, 1.74754122, 1.74754122, 1.74754122,
        1.74570372, 1.74570372, 1.74570372, 1.74570372, 1.74570372,
        1.74576283, 1.74576283, 1.74576283, 1.74576283, 1.74576283]),
 'sp

In [28]:
svd_param = {
    'n_epochs' : 25,
    'lr_all' : 0.01,
    'reg_all' : 0.4
}

knn_param = {
    'k' : 40,
    'sim_options':{
        'name': 'pearson_baseline',
        'min_support' : 1
    },
    'bsl_options': {
        'method': 'als',
        'reg': 1
    }
}

# SVD

In [30]:
algo_svd = SVD(n_epochs=25, lr_all=0.01, reg_all=0.04)
prediction_mf = algo_svd.fit(trainset).test(testset)

prediction_mf

[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=8.232261221953346, details={'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=8.793068205686138, details={'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.747039029339577, details={'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=8.044244967408158, details={'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=9.009328798004315, details={'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.714529028962704, details={'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.310645760283398, details={'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=7.690152874301257, details={'was_impossible': False}),
 Prediction(uid='Vanroy17', iid=58, r_ui=8.0, est=7.683833259305967, details={'was_impossible': False}),
 Prediction(uid='Ammes', iid=814, r_ui=9.0,

In [11]:
recom_svd = algo_svd.predict(uid='Scieska', iid=58)
recom_svd

Prediction(uid='Scieska', iid=58, r_ui=None, est=6.670050959456935, details={'was_impossible': False})

# KNNBaseline

In [33]:
sim_options = {'name': 'pearson_baseline',
                'min_support': 1
               }
bsl_options =  {
        'method': 'als',
        'reg': 1
    }
algo_knn = KNNBaseline(k=40, sim_options=sim_options, bsl_options=bsl_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.603843454168139, details={'actual_k': 21, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=8.990496912392045, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.948602277361147, details={'actual_k': 33, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=8.234402740233417, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=8.294920574152743, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.178838781280222, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.619628850555925, details={'actual_k': 29, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=7.987676040263184, details={'actual_k': 24, 'was_impossible': False}),
 Prediction(uid='Van

In [13]:
recom_knn = algo_knn.predict(uid='Scieska', iid=58)
recom_knn

Prediction(uid='Scieska', iid=58, r_ui=None, est=7.331258408026359, details={'actual_k': 15, 'was_impossible': False})

# Testing Akurasi

In [20]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,1.737142,2.409912,9.962098
BaselineOnly,1.751477,0.640083,0.311103
SlopeOne,1.753658,5.462043,13.869160
KNNWithZScore,1.753755,1.511687,7.022394
KNNWithMeans,1.755588,1.494527,6.984928
CoClustering,1.804534,5.376824,0.402245
SVD,1.810422,20.030613,0.691538
SVDpp,1.846169,1357.928330,21.289238
KNNBasic,1.881817,1.638438,7.590519


In [32]:
df = pd.DataFrame(prediction_knn)
df.to_csv("Prediksi_KNN_weighted.csv")

df = pd.DataFrame(prediction_mf)
df.to_csv("Prediksi_SVD_weighted.csv")